# Необходимо обучить регрессионную модель (предсказание популярности статьи на Хабре) - за бейзлайн возьмите tf-idf + линейная модель:¶

In [3]:
import re
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression

# Препроцессинг

In [ ]:
def remove_punct(s):
    for c in '!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~—«»':
        s = s.replace(c, "")
    return s

In [ ]:
def normalize(s):
    return " ".join((morph.parse(i)[0].normal_form for i in s.lower().split() if len(i) > 3))

In [ ]:
def join_tags(s):
    return ' '.join(remove_punct(s).split())

In [ ]:
# Убиваем линки. 
# Линки бывают разные, поэтому можно по-рахно их размечать в тексте (линк, картинка)
def delete_links(s):
    return re.sub(r"<.*?\>"," link ", s)

In [ ]:
data = pd.read_csv('train_content.csv').dropna()

In [ ]:
data.head()

In [ ]:
data.drop(['_id', 'url', 'date', 'hub', 'png', 'nick', 'name'], axis=1, inplace=True)

In [ ]:
data.head()

In [ ]:
data.reset_index(inplace=True)

In [ ]:
data.head()

In [ ]:
data.drop(['index'], axis=1, inplace=True)

In [ ]:
data.head()

In [ ]:
for c in ['tags', 'title', 'hubs_title', 'description', 'content']:
    if c == 'tags':
        data[c] = data[c].map(join_tags)
    if c == 'content':
        data[c] = data[c].map(delete_links)
    data[c] = data[c].map(remove_punct)
    data[c] = data[c].map(normalize)

In [ ]:
data.head()

In [ ]:
data.to_csv('tokened_data.csv', index=False, encoding='utf-8')

# Загрузка и уменщение размерностей

In [4]:
data = pd.read_csv('tokened_data.csv').dropna()

In [5]:
data.head()

,tags,title,hubs_title,description,content,favs_lognorm
0,eeepc asus эльдорадо,eeepc продажа правда,железо,итак если назад отписаться продажа появиться m...,итак если назад link отписаться link продажа п...,2.484907
1,midnight commander diffview merge filemanager,релиз midnight commander 4705,чёрный дыра,спустя месяц упорный труд выйти новый версия к...,спустя месяц упорный труд выйти новый версия к...,0.000000
2,бизнесмодель бизнесмоделирование,шаг постройка правильный бизнесмодеть,интернетмаркетинг,большинство предприниматель сосредотачиваться ...,link большинство предприниматель сосредотачива...,3.496508
3,python flask mongodb petproject,thunderargs практика использование часть,программирование,история создание часть добрый день вкратце нап...,link история создание link link link часть lin...,3.688879
4,карма usability идея хабра habrahabr,изменение карма пользователь нулевой активность,хабрахабра,назад наконец получить хороший человек долгожд...,назад наконец получить хороший человек долгожд...,0.693147


In [6]:
cut_limit = 130 # отрезать первые {cut_limit} слов от строки
def cut_content(s):
    splitted = s.split()
    if len(splitted) > cut_limit:
        return ' '.join(splitted[:cut_limit])
    return s

In [7]:
data['content'] = data['content'].map(cut_content)

In [8]:
data.head()

,tags,title,hubs_title,description,content,favs_lognorm
0,eeepc asus эльдорадо,eeepc продажа правда,железо,итак если назад отписаться продажа появиться m...,итак если назад link отписаться link продажа п...,2.484907
1,midnight commander diffview merge filemanager,релиз midnight commander 4705,чёрный дыра,спустя месяц упорный труд выйти новый версия к...,спустя месяц упорный труд выйти новый версия к...,0.000000
2,бизнесмодель бизнесмоделирование,шаг постройка правильный бизнесмодеть,интернетмаркетинг,большинство предприниматель сосредотачиваться ...,link большинство предприниматель сосредотачива...,3.496508
3,python flask mongodb petproject,thunderargs практика использование часть,программирование,история создание часть добрый день вкратце нап...,link история создание link link link часть lin...,3.688879
4,карма usability идея хабра habrahabr,изменение карма пользователь нулевой активность,хабрахабра,назад наконец получить хороший человек долгожд...,назад наконец получить хороший человек долгожд...,0.693147


In [9]:
x_train, x_test, y_train, y_test = train_test_split(
    data[data.columns[~data.columns.isin(['favs_lognorm'])]], 
    data['favs_lognorm'],
    test_size=0.19,
    shuffle=True,
    random_state=2200)

# Линейная модель на TfIdf

In [9]:
from scipy.sparse import coo_matrix, hstack
trains = []
tests = []
for c in ['tags', 'title', 'hubs_title', 'description', 'content']:
    vectorizer = TfidfVectorizer(ngram_range=(1,2))
    X_tr = coo_matrix(vectorizer.fit_transform(x_train[c]))
    X_te = coo_matrix(vectorizer.transform(x_test[c]))
    trains.append(X_tr)
    tests.append(X_te)
tfidf_x_train = hstack(trains)
tfidf_x_test = hstack(tests)

In [10]:
clf = make_pipeline(LinearRegression(n_jobs=-1))
clf.fit(tfidf_x_train, y_train.values)
preds = clf.predict(tfidf_x_test)

In [11]:
mean_squared_error(y_test.values, preds) #Baseline score

1.2246443986027111

# Сетка на эмбедингах

In [ ]:
## Эмбеддинг на твиттере (работает хуже)
# !wget http://files.deeppavlov.ai/embeddings/ft_native_300_ru_twitter_nltk_word_tokenize.vec

In [ ]:
## Эмбеддинг на вики и ленте (работает лучше)
# !wget http://files.deeppavlov.ai/embeddings/ft_native_300_ru_wiki_lenta_lemmatize/ft_native_300_ru_wiki_lenta_lemmatize.vec

In [ ]:
## Эмбеддинг на вики и ленте (работает ???? Попробовать)
# !wget http://files.deeppavlov.ai/embeddings/ft_native_300_ru_wiki_lenta_lower_case/ft_native_300_ru_wiki_lenta_lower_case.vec

--2019-05-17 10:17:18--  http://files.deeppavlov.ai/embeddings/ft_native_300_ru_wiki_lenta_lower_case/ft_native_300_ru_wiki_lenta_lower_case.vec
Распознаётся files.deeppavlov.ai (files.deeppavlov.ai)… 93.175.29.74
Подключение к files.deeppavlov.ai (files.deeppavlov.ai)|93.175.29.74|:80... соединение установлено.
HTTP-запрос отправлен. Ожидание ответа… 200 OK
Длина: 4140857273 (3,9G) [application/octet-stream]
Сохранение в: «ft_native_300_ru_wiki_lenta_lower_case.vec»

ft_native_300_ru_wi 100%[===================>]   3,86G  16,3MB/s    за 4m 24s  

2019-05-17 10:21:43 (14,9 MB/s) - «ft_native_300_ru_wiki_lenta_lower_case.vec» сохранён [4140857273/4140857273]



In [10]:
from keras.models import Sequential
from keras.layers import Embedding

In [11]:
# f = open('ft_native_300_ru_twitter_nltk_word_tokenize.vec')
# f = open('ft_native_300_ru_wiki_lenta_lemmatize.vec')
f = open('ft_native_300_ru_wiki_lenta_lower_case.vec')

embedding_values = {}
for line in f:
    value = line.replace('\n','').split(' ')
    word = value[0]
    coef = np.array(value[1:-1],dtype = 'float32')
    embedding_values[word]=coef

In [12]:
def count_max_len(s):
    return len(s.split())

In [13]:
def get_embed_matrix(word_index):
    embedding_matrix = np.zeros((len(word_index)+1, 300),dtype=np.float32)
    sum_finding = 0
    for word, i in word_index.items():
        try:
            embedding_vector = embedding_values[word]
            sum_finding = sum_finding+1
        except:
            embedding_vector = embedding_values["unknown"]
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector
    return embedding_matrix

In [14]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences


X_train = []
X_test = []
embedding_matrix = []
l_word_ind = 0

for c in ['tags', 'title', 'hubs_title', 'description', 'content']:
    maxlen = x_train[c].map(count_max_len).max()
    tokenizer = Tokenizer(num_words=1000000)
    tokenizer.fit_on_texts(x_train[c])
    sequences_train = tokenizer.texts_to_sequences(x_train[c])
    sequences_test = tokenizer.texts_to_sequences(x_test[c])
    pad_train = pad_sequences(sequences_train, maxlen=maxlen)
    pad_test = pad_sequences(sequences_test, maxlen=maxlen)
    X_train.append(pad_train)
    X_test.append(pad_test)
    
    word_index = tokenizer.word_index
    print('Found %s unique tokens.' % len(word_index))
    
    emb_matrix = get_embed_matrix(word_index)
    embedding_matrix.append(emb_matrix)
    
    l_word_ind += len(word_index)+1
    

X_train = np.hstack(X_train)
X_test = np.hstack(X_test)
embedding_matrix = np.vstack(embedding_matrix)

Found 51921 unique tokens.
Found 51037 unique tokens.
Found 411 unique tokens.
Found 84503 unique tokens.
Found 295086 unique tokens.


In [15]:
from __future__ import print_function
import numpy as np
np.random.seed(1000) 

from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Embedding, SpatialDropout1D
from tensorflow.keras.layers import LSTM, SimpleRNN, GRU, Bidirectional
from tensorflow.keras.optimizers import RMSprop, Adam
from sklearn.preprocessing import StandardScaler

ep = 10
batch_size = 256

print('Готовим модель...')
model = Sequential()
model.add(Embedding(
    l_word_ind,
    300,
    weights=[embedding_matrix],
    input_length=X_train.shape[1],
    trainable=False
))

'''
    model.add(SpatialDropout1D(0.22))
    model.add(Bidirectional(SimpleRNN(64)))
    model.add(Dense(1))
    model.add(Activation('linear'))
'''

# model.add(SpatialDropout1D(0.1))
# model.add(Bidirectional(LSTM(32)))
# model.add(Dense(1))
# model.add(Activation('linear'))

model.add(SpatialDropout1D(0.04)) 
model.add(Bidirectional(GRU(64)))
model.add(Dense(1))
model.add(Activation('linear'))


model.compile(
    loss='mean_squared_error', 
    optimizer=Adam(lr=1e-2, clipnorm=4, clipvalue=4), # Взрывается градиент. Градиентклиппинг во исправление ситации.
    metrics=['mean_squared_error'])

print('Обучаем...')
model.fit(
    X_train,
    y_train.values, 
    batch_size=batch_size, 
    epochs=ep, 
    validation_data=(X_test, y_test.values))

score, mse = model.evaluate(X_test, y_test.values,
                            batch_size=batch_size)
print('Тест score:', score)
print('Тест mse:', mse)

Готовим модель...
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Обучаем...
Train on 87420 samples, validate on 20507 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/10
87420/87420 [==============================] - 123s 1ms/sample - loss: 1.7694 - mean_squared_error: 1.7694 - val_loss: 1.6114 - val_mean_squared_error: 1.6114
Epoch 2/10
87420/87420 [==============================] - 122s 1ms/sample - loss: 1.3142 - mean_squared_error: 1.3142 - val_loss: 1.3938 - val_mean_squared_error: 1.3938
Epoch 3/10
87420/87420 [==============================] - 122s 1ms/sample - loss: 1.2344 - mean_squared_error: 1.2344 - val_loss: 1.4362 - val_mean_squared_error: 1.4362
Epoch 4/10
87420/87420 [==============================] - 122s 1ms/sample - loss: 1.2275 - mean_squared_error: 1.2275 - val_loss